In [1]:
import tvm 
import numpy as np

- There often exist several methods to compute the same result, however, different methods will result in different locality and performance. So TVM asks user to provide how to execute the computation called Schedule.

- A Schedule is a set of transformation of computation that transforms the loop of computations in the program.

In [2]:
# declare some variables for use later
n = tvm.var("n")
m = tvm.var("m")

- A schedule can be created from a list of ops, by default the schedule computes tensor in a serial manner in a row-major order. 

In [3]:
# declare a matrix element-wise multiply 
A = tvm.placeholder(shape=(m, n), name="A")
B = tvm.placeholder(shape=(m, n), name="B")
C = tvm.compute(shape=(m, n), 
                fcompute=lambda i, j: A[i, j] * B[i, j], name="C")

In [4]:
s = tvm.create_schedule(ops=[C.op])

lower will transform the computation from definition to the real callable function. With argument `simple_mode=True`, it will return you a readable C like statement

In [5]:
print(tvm.lower(sch=s, args=[A, B, C], simple_mode=True))

produce C {
  for (i, 0, m) {
    for (j, 0, n) {
      C[((i*n) + j)] = (A[((i*n) + j)]*B[((i*n) + j)])
    }
  }
}



- One schedule is composed by multiple stages, and one stage represents schedule for one operation. TVM provides various methods to schedule every stage. 

## Split

`split` can split a specified axis into two axises by `factor`. 

In [6]:
A = tvm.placeholder(shape=(m, ), name="A")
B = tvm.compute(shape=(m, ),
                fcompute=lambda i: A[i] * 2,
                name="B")

In [7]:
s = tvm.create_schedule(B.op)

In [8]:
xo, xi = s[B].split(B.op.axis[0], factor=32)

In [9]:
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

produce B {
  for (i.outer, 0, ((m + 31)/32)) {
    for (i.inner, 0, 32) {
      if (likely(((i.outer*32) < (m - i.inner)))) {
        B[((i.outer*32) + i.inner)] = (A[((i.outer*32) + i.inner)]*2.000000f)
      }
    }
  }
}



You can also split a axis by nparts, which splits the axis contrary with factor

In [10]:
A = tvm.placeholder(shape=(m, ), name="A")
B = tvm.compute(shape=(m, ),
                fcompute=lambda i: A[i] * 2,
                name="B")
s = tvm.create_schedule(ops=B.op)
bx, tx = s[B].split(B.op.axis[0], nparts=32)
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

produce B {
  for (i.outer, 0, 32) {
    for (i.inner, 0, ((m + 31)/32)) {
      if (likely(((i.outer*((m + 31)/32)) < (m - i.inner)))) {
        if (likely(((0 - i.inner) <= (i.outer*((m + 31)/32))))) {
          B[((i.outer*((m + 31)/32)) + i.inner)] = (A[((i.outer*((m + 31)/32)) + i.inner)]*2.000000f)
        }
      }
    }
  }
}



## tile 

`tile` helps execute the computation tile by tile over two axises. 


In [12]:
A = tvm.placeholder(shape=(m, n), name="A")
B = tvm.compute(shape=(m, n), 
                fcompute=lambda i, j: A[i, j], name="B")
s = tvm.create_schedule(B.op)
xo, yo, xi, yi = s[B].tile(B.op.axis[0], B.op.axis[1], x_factor=10, y_factor=5)
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

produce B {
  for (i.outer, 0, ((m + 9)/10)) {
    for (j.outer, 0, ((n + 4)/5)) {
      for (i.inner, 0, 10) {
        for (j.inner, 0, 5) {
          if (likely(((i.outer*10) < (m - i.inner)))) {
            if (likely(((j.outer*5) < (n - j.inner)))) {
              B[(((j.outer*5) + (((i.outer*10) + i.inner)*n)) + j.inner)] = A[(((j.outer*5) + (((i.outer*10) + i.inner)*n)) + j.inner)]
            }
          }
        }
      }
    }
  }
}



## fuse

`fuse` can fuse two consecutive axises of one computation. 

In [13]:
A = tvm.placeholder((m, n), name='A')
B = tvm.compute((m, n), lambda i, j: A[i, j], name='B')

s = tvm.create_schedule(B.op)
# tile to four axises first: (i.outer, j.outer, i.inner, j.inner)
xo, yo, xi, yi = s[B].tile(B.op.axis[0], B.op.axis[1], x_factor=10, y_factor=5)

In [14]:
# then fuse (i.inner, j.inner) into one axis: (i.inner.j.inner.fused)
fused = s[B].fuse(xi, yi)
print(tvm.lower(s, [A, B], simple_mode=True))

produce B {
  for (i.outer, 0, ((m + 9)/10)) {
    for (j.outer, 0, ((n + 4)/5)) {
      for (i.inner.j.inner.fused, 0, 50) {
        if (likely(((i.outer*10) < (m - (i.inner.j.inner.fused/5))))) {
          if (likely(((j.outer*5) < (n - (i.inner.j.inner.fused % 5))))) {
            B[(((j.outer*5) + (i.inner.j.inner.fused % 5)) + (((i.outer*10) + (i.inner.j.inner.fused/5))*n))] = A[(((j.outer*5) + (i.inner.j.inner.fused % 5)) + (((i.outer*10) + (i.inner.j.inner.fused/5))*n))]
          }
        }
      }
    }
  }
}



## reorder 

`reorder` can reorder the axises in the specified order.

In [16]:
A = tvm.placeholder((m, n), name='A')
B = tvm.compute((m, n), lambda i, j: A[i, j], name='B')

s = tvm.create_schedule(B.op)
# tile to four axises first: (i.outer, j.outer, i.inner, j.inner)
xo, yo, xi, yi = s[B].tile(B.op.axis[0], B.op.axis[1], x_factor=10, y_factor=5)
# then reorder the axises: (i.inner, j.outer, i.outer, j.inner)
s[B].reorder(xi, yo, xo, yi)
print(tvm.lower(s, [A, B], simple_mode=True))

produce B {
  for (i.inner, 0, 10) {
    for (j.outer, 0, ((n + 4)/5)) {
      for (i.outer, 0, ((m + 9)/10)) {
        for (j.inner, 0, 5) {
          if (likely((i.inner < (m - (i.outer*10))))) {
            if (likely(((j.outer*5) < (n - j.inner)))) {
              B[(((j.outer*5) + ((i.inner + (i.outer*10))*n)) + j.inner)] = A[(((j.outer*5) + ((i.inner + (i.outer*10))*n)) + j.inner)]
            }
          }
        }
      }
    }
  }
}



## bind
`bind` can bind a specified axis with a thread axis, often used in gpu programming.

In [17]:
A = tvm.placeholder((n,), name='A')
B = tvm.compute(A.shape, lambda i: A[i] * 2, name='B')

s = tvm.create_schedule(B.op)
bx, tx = s[B].split(B.op.axis[0], factor=64)
s[B].bind(bx, tvm.thread_axis("blockIdx.x"))
s[B].bind(tx, tvm.thread_axis("threadIdx.x"))
print(tvm.lower(s, [A, B], simple_mode=True))


produce B {
  // attr [iter_var(blockIdx.x, , blockIdx.x)] thread_extent = ((n + 63)/64)
  // attr [iter_var(threadIdx.x, , threadIdx.x)] thread_extent = 64
  if (likely(((blockIdx.x*64) < (n - threadIdx.x)))) {
    B[((blockIdx.x*64) + threadIdx.x)] = (A[((blockIdx.x*64) + threadIdx.x)]*2.000000f)
  }
}



## compute_at

For a schedule consists of multiple operators, tvm will compute tensors at the root separately by default.

In [18]:
A = tvm.placeholder((m,), name='A')
B = tvm.compute((m,), lambda i: A[i]+1, name='B')
C = tvm.compute((m,), lambda i: B[i]*2, name='C')

s = tvm.create_schedule(C.op)
print(tvm.lower(s, [A, B, C], simple_mode=True))


produce B {
  for (i, 0, m) {
    B[i] = (A[i] + 1.000000f)
  }
}
produce C {
  for (i, 0, m) {
    C[i] = (B[i]*2.000000f)
  }
}



compute_at can move computation of B into the first axis of computation of C.

In [19]:
A = tvm.placeholder((m,), name='A')
B = tvm.compute((m,), lambda i: A[i]+1, name='B')
C = tvm.compute((m,), lambda i: B[i]*2, name='C')

s = tvm.create_schedule(C.op)
s[B].compute_at(s[C], C.op.axis[0])
print(tvm.lower(s, [A, B, C], simple_mode=True))


produce C {
  for (i, 0, m) {
    produce B {
      B[i] = (A[i] + 1.000000f)
    }
    C[i] = (B[i]*2.000000f)
  }
}



## compute_inline

`compute_inline` can mark one stage as inline, then the body of computation will be expanded and inserted at the address where the tensor is required.

In [20]:
A = tvm.placeholder((m,), name='A')
B = tvm.compute((m,), lambda i: A[i]+1, name='B')
C = tvm.compute((m,), lambda i: B[i]*2, name='C')

s = tvm.create_schedule(C.op)
s[B].compute_inline()
print(tvm.lower(s, [A, B, C], simple_mode=True))

produce C {
  for (i, 0, m) {
    C[i] = ((A[i]*2.000000f) + 2.000000f)
  }
}



## compute_root

`compute_root` can move computation of one stage to the root.

In [21]:
A = tvm.placeholder((m,), name='A')
B = tvm.compute((m,), lambda i: A[i]+1, name='B')
C = tvm.compute((m,), lambda i: B[i]*2, name='C')

s = tvm.create_schedule(C.op)
s[B].compute_at(s[C], C.op.axis[0])
s[B].compute_root()
print(tvm.lower(s, [A, B, C], simple_mode=True))


produce B {
  for (i, 0, m) {
    B[i] = (A[i] + 1.000000f)
  }
}
produce C {
  for (i, 0, m) {
    C[i] = (B[i]*2.000000f)
  }
}

